In [1]:
from deepset import *
MODEL = DeepSet(in_features=5, feats=[80,120,70,50,8], n_class=2,pool="mean") #for full dataset Mean pooling, for small dataset Max pooling
model=MODEL
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=model.to(device) 
model

pool:  mean
pool:  mean
pool:  mean
pool:  mean
pool:  mean
pool:  mean


DeepSet(
  (sequential): ModuleList(
    (0): DeepSetLayer(
      (Gamma): Linear(in_features=5, out_features=80, bias=True)
      (Lambda): Linear(in_features=5, out_features=80, bias=True)
      (bn): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    )
    (1): LeakyReLU(negative_slope=0.01)
    (2): DeepSetLayer(
      (Gamma): Linear(in_features=80, out_features=120, bias=True)
      (Lambda): Linear(in_features=80, out_features=120, bias=True)
      (bn): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    )
    (3): LeakyReLU(negative_slope=0.01)
    (4): DeepSetLayer(
      (Gamma): Linear(in_features=120, out_features=70, bias=True)
      (Lambda): Linear(in_features=120, out_features=70, bias=True)
      (bn): BatchNorm1d(70, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    )
    (5): LeakyReLU(negative_slope=0.01)
    (6): DeepSetLayer(
      (Gamma): Linear(in_features=70, out_features=5

In [2]:
import torch
from torch.utils.data import Dataset
import pickle
import os
import numpy as np
import random
import pandas as pd

class StreamingHcaDataset(Dataset): 
    def __init__(self, proton_dir, pion_dir, features=["x", "y", "z", "total_energy", "mean_time"]):
        super().__init__()
        
        self.proton_files = sorted([os.path.join(proton_dir, f) for f in os.listdir(proton_dir) if f.endswith(".parquet")])
        self.pion_files = sorted([os.path.join(pion_dir, f) for f in os.listdir(pion_dir) if f.endswith(".parquet")])

        self.features = features
        self.all_files = self.proton_files + self.pion_files  # Combine file lists
        self.labels = [0] * len(self.proton_files) + [1] * len(self.pion_files)  # 0 for proton, 1 for pion

    def __len__(self):
        return len(self.all_files)  # Total number of files

    def _load_file(self, file_path, label):
        """Loads a single pickle file (containing a single DataFrame) and returns point cloud data with labels."""
        # with open(file_path, "rb") as f:
        #     df = pickle.load(f)  # Load single DataFrame
    
        # df=pd.read_pickle(file_path)
        df=pd.read_parquet(file_path)
        df=df[df["total_energy"]>5]
        
          
        part_feat = df[self.features].to_numpy()

        # Handle NaN and Inf values
        part_feat[np.isnan(part_feat)] = 0.0
        part_feat[np.isinf(part_feat)] = 0.0

        return {
            "part": torch.tensor(part_feat, dtype=torch.float32),
            "label": torch.tensor(label, dtype=torch.long),
            "seq_length": torch.tensor(part_feat.shape[0], dtype=torch.long),
        }

    def __getitem__(self, idx):
        random_idx = random.randint(0, len(self.all_files) - 1)  # Pick a random file
        file_path = self.all_files[random_idx]
        label = self.labels[random_idx]

        return self._load_file(file_path, label)  # Return data from the chosen file


In [3]:
import torch
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    """Custom collate function to handle variable-length point cloud data."""
    parts = [item["part"] for item in batch]  # List of tensors (each of shape [N, 5])
    labels = torch.tensor([item["label"] for item in batch], dtype=torch.long)  # Convert list to tensor
    seq_lengths = torch.tensor([item["seq_length"] for item in batch], dtype=torch.long)  # Convert list to tensor

    # Pad variable-length tensors to the longest sequence in the batch
    padded_parts = pad_sequence(parts, batch_first=True, padding_value=0.0)  # Shape [batch_size, max_seq_len, 5]

    return {"part": padded_parts, "label": labels, "seq_length": seq_lengths}


In [4]:
# positron_dir = "/mnt/newdisk/2003_data_pkl/PKL_poistron/PKL_positron"
# proton_dir = "/mnt/newdisk/2003_data_pkl/PKL_proton/PKL_proton"
positron_dir = "/mnt/c/Users/hnayak/Documents/small_PKL_pion_100GeV_50"
proton_dir = "/mnt/c/Users/hnayak/Documents/small_PKL_proton_100GeV_50"

from torch.utils.data import random_split, DataLoader

# Define dataset
train_dataset = StreamingHcaDataset(proton_dir=proton_dir, pion_dir=positron_dir)

# Define split sizes
total_size = len(train_dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size  # Ensure the sum matches total_size

# Split dataset
train_set, val_set, test_set = random_split(train_dataset, [train_size, val_size, test_size])

# Define DataLoaders
train_loader = DataLoader(train_set, batch_size=32, shuffle=True, collate_fn=collate_fn, num_workers=28)
val_loader = DataLoader(val_set, batch_size=32, shuffle=False, collate_fn=collate_fn,num_workers=28)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, collate_fn=collate_fn,num_workers=28)


print(f"Train: {train_size}, Validation: {val_size}, Test: {test_size}")



Train: 160000, Validation: 20000, Test: 20000


In [5]:
for i,batch in enumerate(train_loader):
    print(i)
    print(batch)
    break

0
{'part': tensor([[[50.0000, 49.0000,  0.0000, 18.5746,  1.3747],
         [49.0000, 49.0000,  0.0000,  6.2649,  1.4144],
         [49.0000, 49.0000,  1.0000, 24.8078,  1.4616],
         ...,
         [50.0000, 50.0000,  6.0000, 11.8740,  1.8301],
         [54.0000, 53.0000,  7.0000,  5.3245,  1.9801],
         [55.0000, 53.0000,  7.0000,  5.7878,  1.9935]],

        [[50.0000, 49.0000,  0.0000, 20.1425,  1.3759],
         [49.0000, 49.0000,  0.0000,  5.5923,  1.4002],
         [49.0000, 49.0000,  1.0000, 28.7575,  1.4679],
         ...,
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

        [[50.0000, 49.0000,  0.0000, 24.1774,  1.3899],
         [50.0000, 49.0000,  1.0000, 32.3433,  1.4565],
         [50.0000, 49.0000,  2.0000, 46.7067,  1.5503],
         ...,
         [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000,  

In [6]:
import tqdm
def test_model(model, test_loader, criterion=None, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    model.eval()  # Set model to evaluation mode

    total_loss = 0.0
    correct = 0
    total_samples = 0

    # Progress bar for testing
    test_loader_tqdm = tqdm(enumerate(test_loader), total=len(test_loader), desc="Testing")

    with torch.no_grad():
        for i,batch in test_loader_tqdm:
            parts = batch["part"].to(device)         # Input point cloud data
            labels = batch["label"].to(device)  # Labels
            batch_size,seq_len,feat_dim=parts.shape
            parts=parts.cpu().numpy().reshape(-1,feat_dim)
            qt = QuantileTransformer(output_distribution='normal', random_state=42)
            parts = qt.fit_transform(parts)
            parts=torch.tensor(parts).reshape(batch_size,seq_len,feat_dim).to(device)

            outputs = model(parts)  # Forward pass
            loss = criterion(outputs, labels) if criterion else 0  # Compute loss if provided

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)  # Get class prediction
            correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)

            test_loader_tqdm.set_postfix(loss=loss.item())  # Update progress bar

    avg_loss = total_loss / len(test_loader) if criterion else 0
    accuracy = correct / total_samples * 100

    print(f"Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")
    return avg_loss , accuracy

In [7]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from tqdm import tqdm  # For progress bar
# from sklearn.preprocessing import QuantileTransformer

# def train_model(model, train_loader, val_loader, num_epochs=10, learning_rate=5e-4, device=None, save_path="./best_model.pth"):
#     if device is None:
#         device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     model.to(device)
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#     best_val_loss = float("inf")  # Initialize best loss

#     for epoch in range(num_epochs):
#         model.train()  # Set model to training mode
#         running_loss = 0.0

#         # Progress bar for training
#         train_loader_tqdm = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")

#         for i, batch in train_loader_tqdm:
#             parts = batch["part"].to(device)         # Input point cloud data
#             batch_size,seq_len,feat_dim=parts.shape
#             parts=parts.cpu().numpy().reshape(-1,feat_dim)
#             qt = QuantileTransformer(output_distribution='normal', random_state=42)
#             parts = qt.fit_transform(parts)
#             parts=torch.tensor(parts).reshape(batch_size,seq_len,feat_dim).to(device)
#             labels = batch["label"].to(device)  # Labels (batch_size,)

#             optimizer.zero_grad()
#             outputs = model(parts)  # Forward pass
#             loss = criterion(outputs, labels)  # Compute loss

#             loss.backward()
#             optimizer.step()

#             running_loss += loss.item()
#             train_loader_tqdm.set_postfix(loss=loss.item())  # Update progress bar

#         avg_train_loss = running_loss / len(train_loader)

#         # Validate the model after each epoch
#         val_loss = test_model(model, val_loader, criterion, device)

#         print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}")

#         # Save model if validation loss improves
#         if val_loss < best_val_loss:
#             best_val_loss = val_loss
#             torch.save(model.state_dict(), save_path)
#             print(f"Model saved at epoch {epoch+1} with val loss {val_loss:.4f}")

#     print("Training complete!")


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import pandas as pd
from sklearn.preprocessing import QuantileTransformer

def train_model(model, train_loader, val_loader, num_epochs=2, learning_rate=5e-4, device=None, save_path="./Models/Z_100GeV_50.pth", log_path="./log_summary_Z_100GeV_50.csv"):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    best_val_loss = float("inf")  # Initialize best loss

    log_data = []  # To store log info for CSV

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        train_loader_tqdm = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}")

        for i, batch in train_loader_tqdm:
            parts = batch["part"].to(device)
            batch_size, seq_len, feat_dim = parts.shape
            parts = parts.cpu().numpy().reshape(-1, feat_dim)
            qt = QuantileTransformer(output_distribution='normal', random_state=42)
            parts = qt.fit_transform(parts)
            parts = torch.tensor(parts).reshape(batch_size, seq_len, feat_dim).float().to(device)

            labels = batch["label"].to(device)

            optimizer.zero_grad()
            outputs = model(parts)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            train_loader_tqdm.set_postfix(loss=loss.item())

        avg_train_loss = running_loss / len(train_loader)
        val_loss, Accuracy = test_model(model, val_loader, criterion, device)
         

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}")

        # Save log
        log_data.append({
            "Epoch": epoch + 1,
            "Train Loss": avg_train_loss,
            "Val Loss": val_loss,
            "Accuracy": Accuracy,  # Placeholder for accuracy
        })

        # Save model if validation improves
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), save_path)
            print(f"Model saved at epoch {epoch+1} with val loss {val_loss:.4f}")

    # Save the log to CSV
    df_log = pd.DataFrame(log_data)
    df_log.to_csv(log_path, index=False)
    print(f"Training log saved to {log_path}")
    print("Training complete!")


In [9]:
device

device(type='cuda')

In [10]:
train_model(model, train_loader,val_loader=val_loader, num_epochs=60, learning_rate=5e-4, device=device)

Testing: 100%|██████████| 625/625 [00:44<00:00, 14.13it/s, loss=0.671]


Test Loss: 0.6765, Accuracy: 57.21%
Epoch [1/60], Train Loss: 0.6918, Val Loss: 0.6765
Model saved at epoch 1 with val loss 0.6765


Testing: 100%|██████████| 625/625 [00:42<00:00, 14.86it/s, loss=0.634]

Test Loss: 0.6595, Accuracy: 60.05%
Epoch [2/60], Train Loss: 0.6663, Val Loss: 0.6595
Model saved at epoch 2 with val loss 0.6595



Testing: 100%|██████████| 625/625 [00:45<00:00, 13.79it/s, loss=0.747]

...

Test Loss: 0.6243, Accuracy: 64.27%
Epoch [58/60], Train Loss: 0.6245, Val Loss: 0.6243



Testing: 100%|██████████| 625/625 [00:46<00:00, 13.32it/s, loss=0.567]

Test Loss: 0.6252, Accuracy: 64.45%
Epoch [59/60], Train Loss: 0.6253, Val Loss: 0.6252



Testing: 100%|██████████| 625/625 [00:46<00:00, 13.36it/s, loss=0.592]

Test Loss: 0.6230, Accuracy: 64.84%
Epoch [60/60], Train Loss: 0.6241, Val Loss: 0.6230
Training log saved to ./log_summary_Z_100GeV_50.csv
Training complete!


In [15]:
model_test = MODEL
model_test.load_state_dict(torch.load("./Models/Z_100GeV_50.pth",weights_only=True))
model_test.to(device)

DeepSet(
  (sequential): ModuleList(
    (0): DeepSetLayer(
      (Gamma): Linear(in_features=5, out_features=80, bias=True)
      (Lambda): Linear(in_features=5, out_features=80, bias=True)
      (bn): BatchNorm1d(80, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    )
    (1): LeakyReLU(negative_slope=0.01)
    (2): DeepSetLayer(
      (Gamma): Linear(in_features=80, out_features=120, bias=True)
      (Lambda): Linear(in_features=80, out_features=120, bias=True)
      (bn): BatchNorm1d(120, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    )
    (3): LeakyReLU(negative_slope=0.01)
    (4): DeepSetLayer(
      (Gamma): Linear(in_features=120, out_features=70, bias=True)
      (Lambda): Linear(in_features=120, out_features=70, bias=True)
      (bn): BatchNorm1d(70, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
    )
    (5): LeakyReLU(negative_slope=0.01)
    (6): DeepSetLayer(
      (Gamma): Linear(in_features=70, out_features=5

In [16]:
total_params = sum(p.numel() for p in MODEL.parameters())
print(total_params)

45292


In [17]:
def evaluate_model(model, data_loader, criterion, device, return_accuracy=False):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    data_loader_tqdm = tqdm(enumerate(data_loader), desc="Testing",total=len(data_loader))
    with torch.no_grad():
        for i,batch in data_loader_tqdm:
            parts = batch["part"].to(device)
            batch_size,seq_len,feat_dim=parts.shape
            parts=parts.cpu().numpy().reshape(-1,feat_dim)
            qt = QuantileTransformer(output_distribution='normal', random_state=42)
            parts = qt.fit_transform(parts)
            parts=torch.tensor(parts).reshape(batch_size,seq_len,feat_dim).to(device)
            labels = batch["label"].to(device)
            
            outputs = model(parts)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            
            if return_accuracy:
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
    
    avg_loss = total_loss / len(data_loader)
    accuracy = 100 * correct / total if return_accuracy else None
    
    return (avg_loss, f"{accuracy}%") if return_accuracy else avg_loss


In [18]:
evaluate_model(model_test, test_loader, nn.CrossEntropyLoss(), device, return_accuracy=True)

Testing: 100%|██████████| 625/625 [01:10<00:00,  8.82it/s]


(0.6210454919338226, '65.155%')